In [1]:
''' Imports '''

import numpy as np 
import pandas as pd
import gizmo_analysis as gizmo  
import utilities as ut
import time as t
import gc

In [2]:
#############################################################################
#constants, binning, & cuts
#############################################################################

MsunToGm = 1.99e33
KpcToCm  = 3.086e21
mp       = 1.67e-24

bin_edge = 30. # out to Rcyl = 30 kpc
bins     = np.arange(-25,25,0.1)
zmax     = 1.5  # include only gas/stars within +/- 1.5 kpc of plane of disk
age_min  = 4    # age minimum in Myr
age_max  = 6    # age maximum in Myr

In [3]:
#########################################################################################################

''' loading and reading the simulation '''

simname = '/scratch/projects/xsede/GalaxiesOnFIRE/fire3/m12i_m7e3/m12i_m7e3_HD_fire3_fireBH_Sep052021_hr_crdiffc690_sdp1e10_gacc31_fa0.5/'                                                         
simdir = '/scratch/projects/xsede/GalaxiesOnFIRE/fire3/m12i_m7e3/m12i_m7e3_HD_fire3_fireBH_Sep052021_hr_crdiffc690_sdp1e10_gacc31_fa0.5/'
part = gizmo.io.Read.read_snapshots(['star','gas'],'snapshot_index', 12
                                    , simulation_name=simname, simulation_directory=simdir,
                                    assign_hosts_rotation=True, assign_hosts=True)

#########################################################################################################



# in utilities.simulation.Snapshot():
* reading:  scratch/projects/xsede/GalaxiesOnFIRE/fire3/m12i_m7e3/m12i_m7e3_HD_fire3_fireBH_Sep052021_hr_crdiffc690_sdp1e10_gacc31_fa0.5/snapshot_times.txt

* reading:  scratch/projects/xsede/GalaxiesOnFIRE/fire3/m12i_m7e3/m12i_m7e3_HD_fire3_fireBH_Sep052021_hr_crdiffc690_sdp1e10_gacc31_fa0.5/snapshot_scalefactors.txt

* reading:  scratch/projects/xsede/GalaxiesOnFIRE/fire3/m12i_m7e3/m12i_m7e3_HD_fire3_fireBH_Sep052021_hr_crdiffc690_sdp1e10_gacc31_fa0.5/snapshot_scale-factors.txt

  using snapshot index = 12, redshift = 9.000


# in gizmo_analysis.gizmo_io.Read():
* reading header from:  scratch/projects/xsede/GalaxiesOnFIRE/fire3/m12i_m7e3/m12i_m7e3_HD_fire3_fireBH_Sep052021_hr_crdiffc690_sdp1e10_gacc31_fa0.5/output/snapshot_012.hdf5
  snapshot contains the following number of particles:
    dark      (id = 1): 70514272 particles
    dark2     (id = 2): 5513331 particles
    gas       (id = 0): 70507868 particles
    star      (id = 4): 6404 part

In [4]:
gas_ids = part['gas']['id']
gas_gen = part['gas']['id.generation']
gas_den = part['gas']['density']
selection = gas_gen == 0
gas_ids_selected = gas_ids[selection]
#dosort = np.argsort(gas_ids_selected)
gas_ids_selected_sorted = np.sort(gas_ids_selected)
gas_den_selected = gas_den[selection]
print(len(gas_ids_selected)) #~7*10^7 cells
#last snapshot has 63837505 gas ids
#snapshot 100 has 69002491 gas ids
#-8.5% length

70507868


In [5]:
snapshot_number = range(13,14)
ids = []
den = []
ids.append(gas_ids_selected)
den.append(gas_den_selected)
new_order= np.zeros(len(gas_ids_selected)) -1


In [7]:
#testing in cell bewlow
for i in range(len(snapshot_number)):
    simname = '/scratch/projects/xsede/GalaxiesOnFIRE/fire3/m12i_m7e3/m12i_m7e3_HD_fire3_fireBH_Sep052021_hr_crdiffc690_sdp1e10_gacc31_fa0.5/'                                                         
    simdir = '/scratch/projects/xsede/GalaxiesOnFIRE/fire3/m12i_m7e3/m12i_m7e3_HD_fire3_fireBH_Sep052021_hr_crdiffc690_sdp1e10_gacc31_fa0.5/'
    part_z = gizmo.io.Read.read_snapshots(['star','gas'],'snapshot_index', snapshot_number[i], simulation_name=simname, simulation_directory=simdir,assign_hosts_rotation=True, assign_hosts=True)
    
    gas_ids_z = part_z['gas']['id']
    gas_gen_z = part_z['gas']['id.generation']
    selection_z = gas_gen_z == 0
    gas_ids_selected_z = gas_ids_z[selection_z]
    gas_ids_selected_sorted = np.sort(gas_ids_selected)
    #locations = dosort[loc]
    
    #new_order[dosort][loc][j] = gas_ids_selected_z
        

    print(len(gas_ids_selected_z))


# in utilities.simulation.Snapshot():
* reading:  scratch/projects/xsede/GalaxiesOnFIRE/fire3/m12i_m7e3/m12i_m7e3_HD_fire3_fireBH_Sep052021_hr_crdiffc690_sdp1e10_gacc31_fa0.5/snapshot_times.txt

* reading:  scratch/projects/xsede/GalaxiesOnFIRE/fire3/m12i_m7e3/m12i_m7e3_HD_fire3_fireBH_Sep052021_hr_crdiffc690_sdp1e10_gacc31_fa0.5/snapshot_scalefactors.txt

* reading:  scratch/projects/xsede/GalaxiesOnFIRE/fire3/m12i_m7e3/m12i_m7e3_HD_fire3_fireBH_Sep052021_hr_crdiffc690_sdp1e10_gacc31_fa0.5/snapshot_scale-factors.txt

  using snapshot index = 13, redshift = 8.775


# in gizmo_analysis.gizmo_io.Read():
* reading header from:  scratch/projects/xsede/GalaxiesOnFIRE/fire3/m12i_m7e3/m12i_m7e3_HD_fire3_fireBH_Sep052021_hr_crdiffc690_sdp1e10_gacc31_fa0.5/output/snapshot_013.hdf5
  snapshot contains the following number of particles:
    dark      (id = 1): 70514272 particles
    dark2     (id = 2): 5513331 particles
    gas       (id = 0): 70506833 particles
    star      (id = 4): 7439 part

In [13]:
gas_ids_z = part_z['gas']['id']
gas_gen_z = part_z['gas']['id.generation']
selection_z = gas_gen_z == 0
gas_ids_selected_z = gas_ids_z[selection_z]
gas_ids_selected_z_sorted = np.sort(gas_ids_selected_z)
dosort = np.argsort(gas_ids_selected_z)
loc = np.searchsorted(gas_ids_selected_z_sorted,gas_ids_selected_sorted)
locations = dosort[loc]


short_new_order = np.array(new_order[len(gas_ids_selected_z):],dtype=int) #short_new_order[locations] #justendings are negative
new_order[locations] = np.concatenate([gas_ids_selected_z,short_new_order]) #put together gas_id_z_selected, so end of new_order[locations] are still -1s
print(new_order[locations])
print(new_order)

#for i in range(len(gas_ids_selected_sorted)):
#    new_order[locations][i] = gas_ids_selected_z_sorted[i]


[ 2.7293291e+07  2.7294891e+07  2.7294357e+07 ... -1.0000000e+00
 -1.0000000e+00 -1.0000000e+00]
[ 3.2912711e+07  3.6035239e+07  3.5054303e+07 ... -1.0000000e+00
 -1.0000000e+00 -1.0000000e+00]


In [ ]:
print(loc)
print(locations)
print(np.where(new_order != -1))
print(new_order)

In [7]:
t0 = t.time()
for i in range(len(snapshot_number)):
    t1 = t.time()
    simname = '/scratch/projects/xsede/GalaxiesOnFIRE/fire3/m12i_m7e3/m12i_m7e3_HD_fire3_fireBH_Sep052021_hr_crdiffc690_sdp1e10_gacc31_fa0.5/'                                                         
    simdir = '/scratch/projects/xsede/GalaxiesOnFIRE/fire3/m12i_m7e3/m12i_m7e3_HD_fire3_fireBH_Sep052021_hr_crdiffc690_sdp1e10_gacc31_fa0.5/'
    partz = gizmo.io.Read.read_snapshots(['star','gas'],'snapshot_index', snapshot_number[i], simulation_name=simname, simulation_directory=simdir,
                                    assign_hosts_rotation=True, assign_hosts=True)
    
    gas_id_z = partz['gas']['id']
    gas_id_z_sorted = np.sort(gas_id_z)
    gas_den_z = partz['gas']['density']
    
    loc = np.searchsorted(gas_ids_selected_sorted,gas_id_z)
    locations = dosort[loc]
    
    ids.append(gas_id_z)
    den.append(gas_den_z)
    
    t2 = t.time()
    print(t2-t1)
t3 = t.time()
print('Total time for all snaps is ' + str(t3-t0))


# in utilities.simulation.Snapshot():
* reading:  scratch/projects/xsede/GalaxiesOnFIRE/fire3/m12i_m7e3/m12i_m7e3_HD_fire3_fireBH_Sep052021_hr_crdiffc690_sdp1e10_gacc31_fa0.5/snapshot_times.txt

* reading:  scratch/projects/xsede/GalaxiesOnFIRE/fire3/m12i_m7e3/m12i_m7e3_HD_fire3_fireBH_Sep052021_hr_crdiffc690_sdp1e10_gacc31_fa0.5/snapshot_scalefactors.txt

* reading:  scratch/projects/xsede/GalaxiesOnFIRE/fire3/m12i_m7e3/m12i_m7e3_HD_fire3_fireBH_Sep052021_hr_crdiffc690_sdp1e10_gacc31_fa0.5/snapshot_scale-factors.txt

  using snapshot index = 101, redshift = 2.273


# in gizmo_analysis.gizmo_io.Read():
* reading header from:  scratch/projects/xsede/GalaxiesOnFIRE/fire3/m12i_m7e3/m12i_m7e3_HD_fire3_fireBH_Sep052021_hr_crdiffc690_sdp1e10_gacc31_fa0.5/output/snapshot_101.hdf5
  snapshot contains the following number of particles:
    dark      (id = 1): 70514272 particles
    dark2     (id = 2): 5513331 particles
    gas       (id = 0): 69047036 particles
    star      (id = 4): 1471241 

In [ ]:
#115.67876434326172s with matching
#94.85135626792908s without matching

In [ ]:
new_order[dosort[loc][i]] = gas_ids_selected_z[i]
print(new_order)
print(gas_ids_selected_z)

In [ ]:
print(gas_ids_selected[locations])
print(gas_ids_selected_z)
print(gas_ids_selected_z[19394])
print(gas_ids_selected[locations][19394])

In [ ]:
test = np.zeros(10)-1
test[np.array([1, 2, 3])] = [3, 7, 1]
print(test)